In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [47]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 01:00:00+00:00


In [48]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [49]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-03 19:38:33,720 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:38:33,723 INFO: Initializing external client
2025-03-03 19:38:33,723 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:38:34,325 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652


In [50]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.94s) 


In [51]:
df['pickup_hour']

0     2025-03-04 00:00:00+00:00
1     2025-03-04 00:00:00+00:00
2     2025-03-04 00:00:00+00:00
3     2025-03-04 00:00:00+00:00
4     2025-03-04 00:00:00+00:00
                 ...           
497   2025-03-04 01:00:00+00:00
498   2025-03-04 01:00:00+00:00
499   2025-03-04 01:00:00+00:00
500   2025-03-04 01:00:00+00:00
501   2025-03-04 01:00:00+00:00
Name: pickup_hour, Length: 502, dtype: datetime64[us, Etc/UTC]

In [52]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-03 19:38:43,136 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:38:43,139 INFO: Initializing external client
2025-03-03 19:38:43,139 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:38:43,812 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652
Current UTC time: 2025-03-04 00:38:43.136464+00:00
Next hour: 2025-03-04 01:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 01:00
Found 251 records


In [53]:
now = datetime.now(timezone.utc)

In [54]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,221,0.0,2025-03-04 01:00:00+00:00
252,122,0.0,2025-03-04 01:00:00+00:00
253,63,0.0,2025-03-04 01:00:00+00:00
254,91,0.0,2025-03-04 01:00:00+00:00
255,62,1.0,2025-03-04 01:00:00+00:00
...,...,...,...
497,96,0.0,2025-03-04 01:00:00+00:00
498,247,1.0,2025-03-04 01:00:00+00:00
499,120,0.0,2025-03-04 01:00:00+00:00
500,261,16.0,2025-03-04 01:00:00+00:00


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  502 non-null    int32                  
 1   predicted_demand    502 non-null    float64                
 2   pickup_hour         502 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 9.9 KB


In [56]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [57]:
dt = current_date.ceil('h')


In [59]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-03 19:39:55,633 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:39:55,634 INFO: Initializing external client
2025-03-03 19:39:55,635 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:39:56,394 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.42s) 


In [60]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 


,pickup_location_id,predicted_demand,pickup_hour


In [61]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,150,0.0,2025-03-04 00:00:00+00:00
1,131,0.0,2025-03-04 00:00:00+00:00
2,74,14.0,2025-03-04 00:00:00+00:00
3,253,0.0,2025-03-04 00:00:00+00:00
4,201,0.0,2025-03-04 00:00:00+00:00
...,...,...,...
497,96,0.0,2025-03-04 01:00:00+00:00
498,247,1.0,2025-03-04 01:00:00+00:00
499,120,0.0,2025-03-04 01:00:00+00:00
500,261,16.0,2025-03-04 01:00:00+00:00


In [62]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [63]:
predictions = fetch_next_hour_predictions()

2025-03-03 19:40:06,802 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:40:06,806 INFO: Initializing external client
2025-03-03 19:40:06,806 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:40:07,356 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652
Current UTC time: 2025-03-04 00:40:06.802113+00:00
Next hour: 2025-03-04 01:00:00+00:00
Found 251 records


In [64]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-03 19:40:09,111 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:40:09,113 INFO: Initializing external client
2025-03-03 19:40:09,114 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:40:10,038 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652


In [65]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.92s) 


In [66]:
df["pickup_hour"].max()

Timestamp('2025-03-04 01:00:00+0000', tz='Etc/UTC')

In [67]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 01:00:00+00:00


In [68]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
251,221,0.0,2025-03-04 01:00:00+00:00
252,122,0.0,2025-03-04 01:00:00+00:00
253,63,0.0,2025-03-04 01:00:00+00:00
254,91,0.0,2025-03-04 01:00:00+00:00
255,62,1.0,2025-03-04 01:00:00+00:00
...,...,...,...
497,96,0.0,2025-03-04 01:00:00+00:00
498,247,1.0,2025-03-04 01:00:00+00:00
499,120,0.0,2025-03-04 01:00:00+00:00
500,261,16.0,2025-03-04 01:00:00+00:00
